# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 2880
Session ID: 46a09194-5de6-4911-b44d-a6943faae05e
Applying the following default arguments:
--glue_kernel_version 1.0.8
--enable-glue-datacatalog true
Waiting for session 46a09194-5de6-4911-b44d-a6943faae05e to get into ready status...
Session 46a09194-5de6-4911-b44d-a6943faae05e ha

In [2]:
# Read from the customers table in the glue data catalog using a dynamic frame
dynamicFrameCustomers = glueContext.create_dynamic_frame.from_catalog(
database = "pyspark_tutorial_db", 
table_name = "customers"
)

# Show the top 10 rows from the dynamic dataframe
dynamicFrameCustomers.show(10)

{"customerid": 293, "firstname": "Catherine", "lastname": "Abel", "fullname": "Catherine Abel"}
{"customerid": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie"}
{"customerid": 297, "firstname": "Humberto", "lastname": "Acevedo", "fullname": "Humberto Acevedo"}
{"customerid": 291, "firstname": "Gustavo", "lastname": "Achong", "fullname": "Gustavo Achong"}
{"customerid": 299, "firstname": "Pilar", "lastname": "Ackerman", "fullname": "Pilar Ackerman"}
{"customerid": 305, "firstname": "Carla", "lastname": "Adams", "fullname": "Carla Adams"}
{"customerid": 301, "firstname": "Frances", "lastname": "Adams", "fullname": "Frances Adams"}
{"customerid": 307, "firstname": "Jay", "lastname": "Adams", "fullname": "Jay Adams"}
{"customerid": 309, "firstname": "Ronald", "lastname": "Adina", "fullname": "Ronald Adina"}
{"customerid": 311, "firstname": "Samuel", "lastname": "Agcaoili", "fullname": "Samuel Agcaoili"}


In [4]:
dynamicFrameCustomers.printSchema()

root
|-- customerid: long
|-- firstname: string
|-- lastname: string
|-- fullname: string


In [5]:
# count the dynamic frame
dynamicFrameCustomers.count()

635


In [9]:
dyfCustomerSelectFields = dynamicFrameCustomers.select_fields(['customerid','fullname']).show(10)

{"customerid": 293, "fullname": "Catherine Abel"}
{"customerid": 295, "fullname": "Kim Abercrombie"}
{"customerid": 297, "fullname": "Humberto Acevedo"}
{"customerid": 291, "fullname": "Gustavo Achong"}
{"customerid": 299, "fullname": "Pilar Ackerman"}
{"customerid": 305, "fullname": "Carla Adams"}
{"customerid": 301, "fullname": "Frances Adams"}
{"customerid": 307, "fullname": "Jay Adams"}
{"customerid": 309, "fullname": "Ronald Adina"}
{"customerid": 311, "fullname": "Samuel Agcaoili"}


In [11]:
dynamicFrameCustomers.select_fields(['customerid','fullname']).show(10)

{"customerid": 293, "fullname": "Catherine Abel"}
{"customerid": 295, "fullname": "Kim Abercrombie"}
{"customerid": 297, "fullname": "Humberto Acevedo"}
{"customerid": 291, "fullname": "Gustavo Achong"}
{"customerid": 299, "fullname": "Pilar Ackerman"}
{"customerid": 305, "fullname": "Carla Adams"}
{"customerid": 301, "fullname": "Frances Adams"}
{"customerid": 307, "fullname": "Jay Adams"}
{"customerid": 309, "fullname": "Ronald Adina"}
{"customerid": 311, "fullname": "Samuel Agcaoili"}


In [13]:
# drop fields from dynamic dataframe
dyfCustomerDropFields= dynamicFrameCustomers.drop_fields(['firstname','lastname'])

dyfCustomerDropFields.show(10)

{"customerid": 293, "fullname": "Catherine Abel"}
{"customerid": 295, "fullname": "Kim Abercrombie"}
{"customerid": 297, "fullname": "Humberto Acevedo"}
{"customerid": 291, "fullname": "Gustavo Achong"}
{"customerid": 299, "fullname": "Pilar Ackerman"}
{"customerid": 305, "fullname": "Carla Adams"}
{"customerid": 301, "fullname": "Frances Adams"}
{"customerid": 307, "fullname": "Jay Adams"}
{"customerid": 309, "fullname": "Ronald Adina"}
{"customerid": 311, "fullname": "Samuel Agcaoili"}


In [15]:
# mapping full name -> name
mapping = [('customerid','long', 'customerid', 'long'), ('fullname','string','name','string')]

dfyMapping = ApplyMapping.apply(
    frame = dyfCustomerDropFields,
    mappings = mapping,
    transformation_ctx = 'applymapping1'
)

dfyMapping.show(10)

{"customerid": 293, "name": "Catherine Abel"}
{"customerid": 295, "name": "Kim Abercrombie"}
{"customerid": 297, "name": "Humberto Acevedo"}
{"customerid": 291, "name": "Gustavo Achong"}
{"customerid": 299, "name": "Pilar Ackerman"}
{"customerid": 305, "name": "Carla Adams"}
{"customerid": 301, "name": "Frances Adams"}
{"customerid": 307, "name": "Jay Adams"}
{"customerid": 309, "name": "Ronald Adina"}
{"customerid": 311, "name": "Samuel Agcaoili"}


In [16]:
dyfFilter = Filter.apply(
    frame = dynamicFrameCustomers,
    f = lambda x: x['lastname'] in 'Adams'
)
dyfFilter.show()

{"lastname": "Adams", "firstname": "Carla", "customerid": 305, "fullname": "Carla Adams"}
{"lastname": "Adams", "firstname": "Frances", "customerid": 301, "fullname": "Frances Adams"}
{"lastname": "Adams", "firstname": "Jay", "customerid": 307, "fullname": "Jay Adams"}


In [17]:
# Read from the customers table in the glue data catalog using a dynamic frame
dynamicFrameOrders = glueContext.create_dynamic_frame.from_catalog(
database = "pyspark_tutorial_db", 
table_name = "orders"
)

# show top 10 rows of orders table
dynamicFrameOrders.show(10)

{"salesorderid": 43659, "salesorderdetailid": 1, "orderdate": "5/31/2011", "duedate": "6/12/2011", "shipdate": "6/7/2011", "employeeid": 279, "customerid": 1045, "subtotal": 20565.6206, "taxamt": 1971.5149, "freight": 616.0984, "totaldue": 23153.2339, "productid": 776, "orderqty": 1, "unitprice": 2024.9940, "unitpricediscount": 0.0000, "linetotal": 2024.9940}
{"salesorderid": 43659, "salesorderdetailid": 2, "orderdate": "5/31/2011", "duedate": "6/12/2011", "shipdate": "6/7/2011", "employeeid": 279, "customerid": 1045, "subtotal": 20565.6206, "taxamt": 1971.5149, "freight": 616.0984, "totaldue": 23153.2339, "productid": 777, "orderqty": 3, "unitprice": 2024.9940, "unitpricediscount": 0.0000, "linetotal": 6074.9820}
{"salesorderid": 43659, "salesorderdetailid": 3, "orderdate": "5/31/2011", "duedate": "6/12/2011", "shipdate": "6/7/2011", "employeeid": 279, "customerid": 1045, "subtotal": 20565.6206, "taxamt": 1971.5149, "freight": 616.0984, "totaldue": 23153.2339, "productid": 778, "order

In [18]:
dyfjoin = dynamicFrameCustomers.join(['customerid'],['customerid'], dynamicFrameOrders)

dyfjoin.show(10)

{"freight": 94.1433, "subtotal": 3170.6280, "salesorderdetailid": 39158, "productid": 783, "linetotal": 2753.9880, "employeeid": 288, ".customerid": 419, "taxamt": 301.2584, "salesorderid": 51690, "duedate": "7/12/2013", "orderqty": 2, "shipdate": "7/7/2013", "lastname": "Bartness", "firstname": "David", "totaldue": 3566.0297, "unitprice": 1376.9940, "orderdate": "6/30/2013", "unitpricediscount": 0.0000, "customerid": 419, "fullname": "David Bartness"}
{"freight": 94.1433, "subtotal": 3170.6280, "salesorderdetailid": 39159, "productid": 907, "linetotal": 63.9000, "employeeid": 288, ".customerid": 419, "taxamt": 301.2584, "salesorderid": 51690, "duedate": "7/12/2013", "orderqty": 1, "shipdate": "7/7/2013", "lastname": "Bartness", "firstname": "David", "totaldue": 3566.0297, "unitprice": 63.9000, "orderdate": "6/30/2013", "unitpricediscount": 0.0000, "customerid": 419, "fullname": "David Bartness"}
{"freight": 94.1433, "subtotal": 3170.6280, "salesorderdetailid": 39160, "productid": 867,

In [19]:
# write down the data in a Dynamic Frame to S3 location. 
glueContext.write_dynamic_frame.from_options(
    frame = dynamicFrameCustomers,
    connection_type = 's3',
    connection_options = {'path': 's3://virgilio-murillo-pyspark-tutorial/dynamic-dataframe/'},
    format = 'csv',
    format_options = {
        'separator': ','
    },
    transformation_ctx = 'datasink2'
)

In [22]:
# write down to data catalog using a dynamic frame
glueContext.write_dynamic_frame.from_catalog(
    frame = dynamicFrameCustomers,
    database = 'pyspark_tutorial_db',
    table_name = 'customers_write_dyf',
)

In [24]:
sparkDf = dynamicFrameCustomers.toDF()

sparkDf.show()

+----------+----------+-----------+--------------------+
|customerid| firstname|   lastname|            fullname|
+----------+----------+-----------+--------------------+
|       293| Catherine|       Abel|      Catherine Abel|
|       295|       Kim|Abercrombie|     Kim Abercrombie|
|       297|  Humberto|    Acevedo|    Humberto Acevedo|
|       291|   Gustavo|     Achong|      Gustavo Achong|
|       299|     Pilar|   Ackerman|      Pilar Ackerman|
|       305|     Carla|      Adams|         Carla Adams|
|       301|   Frances|      Adams|       Frances Adams|
|       307|       Jay|      Adams|           Jay Adams|
|       309|    Ronald|      Adina|        Ronald Adina|
|       311|    Samuel|   Agcaoili|     Samuel Agcaoili|
|       313|     James|    Aguilar|       James Aguilar|
|       315|    Robert|   Ahlering|     Robert Ahlering|
|       319|       Kim|      Akers|           Kim Akers|
|       441|   Stanley|       Alan|        Stanley Alan|
|       323|       Amy|    Albe

In [25]:
dfSelect = sparkDf.select('customerid','fullname')

dfSelect.show()

+----------+--------------------+
|customerid|            fullname|
+----------+--------------------+
|       293|      Catherine Abel|
|       295|     Kim Abercrombie|
|       297|    Humberto Acevedo|
|       291|      Gustavo Achong|
|       299|      Pilar Ackerman|
|       305|         Carla Adams|
|       301|       Frances Adams|
|       307|           Jay Adams|
|       309|        Ronald Adina|
|       311|     Samuel Agcaoili|
|       313|       James Aguilar|
|       315|     Robert Ahlering|
|       319|           Kim Akers|
|       441|        Stanley Alan|
|       323|         Amy Alberts|
|       325|       Anna Albright|
|       327|       Milton Albury|
|       329|         Paul Alcorn|
|       331|    Gregory Alderson|
|       333|J. Phillip Alexander|
+----------+--------------------+
only showing top 20 rows


In [26]:
# import the lit lib
from pyspark.sql.functions import lit

# add new column with literal value
dfNewColumn = sparkDf.withColumn('date',lit('2022-7-25'))

dfNewColumn.show()

+----------+----------+-----------+--------------------+---------+
|customerid| firstname|   lastname|            fullname|     date|
+----------+----------+-----------+--------------------+---------+
|       293| Catherine|       Abel|      Catherine Abel|2022-7-25|
|       295|       Kim|Abercrombie|     Kim Abercrombie|2022-7-25|
|       297|  Humberto|    Acevedo|    Humberto Acevedo|2022-7-25|
|       291|   Gustavo|     Achong|      Gustavo Achong|2022-7-25|
|       299|     Pilar|   Ackerman|      Pilar Ackerman|2022-7-25|
|       305|     Carla|      Adams|         Carla Adams|2022-7-25|
|       301|   Frances|      Adams|       Frances Adams|2022-7-25|
|       307|       Jay|      Adams|           Jay Adams|2022-7-25|
|       309|    Ronald|      Adina|        Ronald Adina|2022-7-25|
|       311|    Samuel|   Agcaoili|     Samuel Agcaoili|2022-7-25|
|       313|     James|    Aguilar|       James Aguilar|2022-7-25|
|       315|    Robert|   Ahlering|     Robert Ahlering|2022-7

In [27]:
# create a new column and add two strings together
from pyspark.sql.functions import concat

dfnewFullname = sparkDf.withColumn('new_full_name',concat('firstname',concat(lit(' '), 'lastname')))

dfnewFullname.show()

+----------+----------+-----------+--------------------+--------------------+
|customerid| firstname|   lastname|            fullname|       new_full_name|
+----------+----------+-----------+--------------------+--------------------+
|       293| Catherine|       Abel|      Catherine Abel|      Catherine Abel|
|       295|       Kim|Abercrombie|     Kim Abercrombie|     Kim Abercrombie|
|       297|  Humberto|    Acevedo|    Humberto Acevedo|    Humberto Acevedo|
|       291|   Gustavo|     Achong|      Gustavo Achong|      Gustavo Achong|
|       299|     Pilar|   Ackerman|      Pilar Ackerman|      Pilar Ackerman|
|       305|     Carla|      Adams|         Carla Adams|         Carla Adams|
|       301|   Frances|      Adams|       Frances Adams|       Frances Adams|
|       307|       Jay|      Adams|           Jay Adams|           Jay Adams|
|       309|    Ronald|      Adina|        Ronald Adina|        Ronald Adina|
|       311|    Samuel|   Agcaoili|     Samuel Agcaoili|     Sam

In [28]:
# drop columns
dfDropCol = sparkDf.drop('firstname','lastname')

dfDropCol.show()

+----------+--------------------+
|customerid|            fullname|
+----------+--------------------+
|       293|      Catherine Abel|
|       295|     Kim Abercrombie|
|       297|    Humberto Acevedo|
|       291|      Gustavo Achong|
|       299|      Pilar Ackerman|
|       305|         Carla Adams|
|       301|       Frances Adams|
|       307|           Jay Adams|
|       309|        Ronald Adina|
|       311|     Samuel Agcaoili|
|       313|       James Aguilar|
|       315|     Robert Ahlering|
|       319|           Kim Akers|
|       441|        Stanley Alan|
|       323|         Amy Alberts|
|       325|       Anna Albright|
|       327|       Milton Albury|
|       329|         Paul Alcorn|
|       331|    Gregory Alderson|
|       333|J. Phillip Alexander|
+----------+--------------------+
only showing top 20 rows


In [29]:
# renaming columns
dfRenameCols = sparkDf.withColumnRenamed('fullname','full_name_new').show()

+----------+----------+-----------+--------------------+
|customerid| firstname|   lastname|       full_name_new|
+----------+----------+-----------+--------------------+
|       293| Catherine|       Abel|      Catherine Abel|
|       295|       Kim|Abercrombie|     Kim Abercrombie|
|       297|  Humberto|    Acevedo|    Humberto Acevedo|
|       291|   Gustavo|     Achong|      Gustavo Achong|
|       299|     Pilar|   Ackerman|      Pilar Ackerman|
|       305|     Carla|      Adams|         Carla Adams|
|       301|   Frances|      Adams|       Frances Adams|
|       307|       Jay|      Adams|           Jay Adams|
|       309|    Ronald|      Adina|        Ronald Adina|
|       311|    Samuel|   Agcaoili|     Samuel Agcaoili|
|       313|     James|    Aguilar|       James Aguilar|
|       315|    Robert|   Ahlering|     Robert Ahlering|
|       319|       Kim|      Akers|           Kim Akers|
|       441|   Stanley|       Alan|        Stanley Alan|
|       323|       Amy|    Albe

In [31]:
# group on last name and count the number of times they appear
sparkDf.groupby('lastname').count().show()

+-----------+-----+
|   lastname|count|
+-----------+-----+
|Abercrombie|    1|
|     Achong|    1|
|      Adams|    3|
|  Alexander|    3|
|      Allen|    3|
| Altamirano|    1|
|     Alvaro|    1|
|  Appelbaum|    1|
|    Arakawa|    1|
|       Ault|    1|
|     Bailey|    1|
|    Baldwin|    1|
|      Bales|    1|
|     Barker|    1|
|     Beaver|    1|
|       Beck|    1|
|      Belli|    1|
|  Bernacchi|    1|
|    Blanton|    1|
|    Bradley|    1|
+-----------+-----+
only showing top 20 rows


In [32]:
# Filter spark DataFrame for customers who have the last name Adams
sparkDf.filter(sparkDf["lastname"] == "Adams").show()

+----------+---------+--------+-------------+
|customerid|firstname|lastname|     fullname|
+----------+---------+--------+-------------+
|       305|    Carla|   Adams|  Carla Adams|
|       301|  Frances|   Adams|Frances Adams|
|       307|      Jay|   Adams|    Jay Adams|
+----------+---------+--------+-------------+


In [33]:
# Where clause spark DataFrame for customers who have the last name Adams
sparkDf.where("lastname =='Adams'").show()

+----------+---------+--------+-------------+
|customerid|firstname|lastname|     fullname|
+----------+---------+--------+-------------+
|       305|    Carla|   Adams|  Carla Adams|
|       301|  Frances|   Adams|Frances Adams|
|       307|      Jay|   Adams|    Jay Adams|
+----------+---------+--------+-------------+


In [34]:
# Read from the customers table in the glue data catalog using a dynamic frame and convert to spark dataframe
dfOrders = glueContext.create_dynamic_frame.from_catalog(
                                        database = "pyspark_tutorial_db", 
                                        table_name = "orders"
                                    ).toDF()

In [35]:
# Inner Join Customers Spark DF to Orders Spark DF
sparkDf.join(dfOrders,sparkDf.customerid ==  dfOrders.customerid,"inner").show(truncate=False)

+----------+---------+---------+---------------+------------+------------------+---------+---------+--------+----------+----------+----------+---------+--------+----------+---------+--------+---------+-----------------+---------+
|customerid|firstname|lastname |fullname       |salesorderid|salesorderdetailid|orderdate|duedate  |shipdate|employeeid|customerid|subtotal  |taxamt   |freight |totaldue  |productid|orderqty|unitprice|unitpricediscount|linetotal|
+----------+---------+---------+---------------+------------+------------------+---------+---------+--------+----------+----------+----------+---------+--------+----------+---------+--------+---------+-----------------+---------+
|721       |Takiko   |Collins  |Takiko Collins |43660       |13                |5/31/2011|6/12/2011|6/7/2011|279       |721       |1294.2529 |124.2483 |38.8276 |1457.3288 |762      |1       |419.4589 |0.0000           |419.4589 |
|721       |Takiko   |Collins  |Takiko Collins |43660       |14                |

In [36]:
#left join on orders and adams df
dfOrders.join(dfAdams,dfAdams.customerid ==  dfOrders.customerid,"left").show(100)

NameError: name 'dfAdams' is not defined


In [37]:
# Import Dynamic DataFrame class
from awsglue.dynamicframe import DynamicFrame

#Convert from Spark Data Frame to Glue Dynamic Frame
dyfCustomersConvert = DynamicFrame.fromDF(sparkDf, glueContext, "convert")

#Show converted Glue Dynamic Frame
dyfCustomersConvert.show()

{"customerid": 293, "firstname": "Catherine", "lastname": "Abel", "fullname": "Catherine Abel"}
{"customerid": 295, "firstname": "Kim", "lastname": "Abercrombie", "fullname": "Kim Abercrombie"}
{"customerid": 297, "firstname": "Humberto", "lastname": "Acevedo", "fullname": "Humberto Acevedo"}
{"customerid": 291, "firstname": "Gustavo", "lastname": "Achong", "fullname": "Gustavo Achong"}
{"customerid": 299, "firstname": "Pilar", "lastname": "Ackerman", "fullname": "Pilar Ackerman"}
{"customerid": 305, "firstname": "Carla", "lastname": "Adams", "fullname": "Carla Adams"}
{"customerid": 301, "firstname": "Frances", "lastname": "Adams", "fullname": "Frances Adams"}
{"customerid": 307, "firstname": "Jay", "lastname": "Adams", "fullname": "Jay Adams"}
{"customerid": 309, "firstname": "Ronald", "lastname": "Adina", "fullname": "Ronald Adina"}
{"customerid": 311, "firstname": "Samuel", "lastname": "Agcaoili", "fullname": "Samuel Agcaoili"}
{"customerid": 313, "firstname": "James", "lastname": 

In [40]:
# write down the data in converted Dynamic Frame to S3 location. 
glueContext.write_dynamic_frame.from_options(
                            frame = dyfCustomersConvert,
                            connection_type="s3", 
                            connection_options = {"path": "s3://virgilio-murillo-pyspark-tutorial/write_down_dyf_to_s3"}, 
                            format = "csv", 
                            format_options={
                                "separator": ","
                                },
                            transformation_ctx = "datasink2")

In [41]:
# write data from the converted to customers_write_dyf table using the meta data stored in the glue data catalog 
glueContext.write_dynamic_frame.from_catalog(
    frame = dyfCustomersConvert,
    database = "pyspark_tutorial_db",  
    table_name = "customers_write_dyf")